# Handwritten Text Recognition Using Tesseract OCR

This notebook demonstrates how to build a simple GUI application using Python for recognizing handwritten text from images using Tesseract OCR. It includes image preprocessing, text extraction, and saving the recognized text into a PDF file.


In [4]:
# Import required libraries
from tkinter import *  # Tkinter for GUI components
from tkinter import filedialog  # To use the file dialog for file selection
import fpdf  # To create PDFs from recognized text
import cv2  # OpenCV for image processing
import numpy as np  # For numerical operations
from PIL import Image  # PIL for image handling
import pytesseract  # OCR library

# Set the path to Tesseract executable; ensure Tesseract is installed on your system
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Global variable to hold the recognized text
result = None


## Function to Browse and Select an Image

The `browseFiles()` function allows the user to select an image file. The function processes the selected image, applies noise removal, and extracts text using Tesseract OCR.


In [6]:
def browseFiles():
    """
    Function to browse and select an image file.
    It processes the selected image and extracts text using Tesseract OCR.
    """
    # File types to filter images
    py = r"*.png *.jpg *.jpeg"
    
    # Open file dialog to select an image
    filename = filedialog.askopenfilename(initialdir="/", title="Select a File",
                                          filetypes=(("images", py), ("all files", "*.*")))
    
    # Check if the user cancels file selection
    if filename == "":
        return
    
    # Read image using OpenCV
    img = cv2.imread(filename)

    # Convert image to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)

    # Save the processed image (optional, for debugging purposes)
    cv2.imwrite("removed_noise.png", img)

    # Recognize text from the processed image using Tesseract
    global result
    result = pytesseract.image_to_string(Image.open(filename))

    # Display recognized text on the GUI
    label_file_explorer.configure(text=result)


## Function to Convert Text to PDF

The `pdf()` function converts the recognized text into a PDF file and saves it as `converted.pdf`.


In [7]:
def pdf():
    """
    Function to convert recognized text into a PDF.
    """
    global result
    
    # Create a new PDF document
    pdf = fpdf.FPDF(format='letter')
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.write(5, result)  # Write the recognized text to the PDF
    pdf.ln()  # Add a new line
    pdf.output("converted.pdf")  # Save the PDF as 'converted.pdf'


## Setting Up the Main GUI Window

This section sets up the GUI window using Tkinter, including buttons for browsing files and converting text to PDF, and displays the recognized text.


In [8]:
# Set up the main GUI window
window = Tk()
window.title('Handwritten Text Recognition')  # Set window title
window.geometry("700x350")  # Set window size
window.config(background="white")  # Set background color

# Label to display the header
reg_info = Label(window, text="Handwritten Text Recognition Using Pytesseract", 
                 width='80', height='2', font=("Arial", 12, "bold"), fg="black", bg='lightgrey')
reg_info.place(x=370, y=18, anchor='center')

# Label to display the recognized text
label_file_explorer = Label(window, text="See the Output Here", font=("Arial", 10, "bold"),
                            width=90, height=12, fg="blue")
label_file_explorer.place(x=0, y=35)

# Button to browse files
button_explore = Button(window, text="Browse Files", fg="white", bg="black", 
                        font=("Arial", 10, "bold"), width=10, command=browseFiles)
button_explore.place(x=250, y=270)

# Label indicating to select an image
text = Label(window, text="(Select an image)", bg="white", fg="black", 
             font=("Arial", 8, "bold"))
text.place(x=242, y=300)

# Button to convert text to PDF
button1 = Button(window, text="Convert Text to PDF", fg="white", bg="black", 
                 font=("Arial", 10, "bold"), width=15, command=pdf)
button1.place(x=370, y=270)

# Run the GUI event loop
window.mainloop()


## Text Comparison and Accuracy Check

The final section compares the recognized text with a predefined sample text and calculates the similarity percentage to estimate the accuracy of the OCR.


In [9]:
# Text comparison and accuracy check
from difflib import SequenceMatcher

# Check if there is recognized text available for comparison
if result is not None:
    # Sample text to compare with recognized text (can be replaced with your ground truth)
    s = "We start With good\n\nBecause all businesses should\n\nbe doing something good"
    s1 = result

    # Function to calculate similarity between two texts
    def similar(a, b):
        return "\nThe accuracy of the model is " + str(SequenceMatcher(None, a, b).ratio() * 100) + "%\n"

    # Print the accuracy of the OCR result compared to the sample text
    print(similar(s, s1))
    result = None



The accuracy of the model is 83.44370860927152%

